This is a live notebook with experimental code to develop analysis investigating the correlation between features.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'


['debug_windowing_data.ipynb', 'testing_mlpwrapper.ipynb', 'logs', 'analyse_results.ipynb', 'gait_data_features.ipynb', 'feature_development.ipynb', 'data', 'feature_analysis.ipynb', 'test_sample_entropy_implementations.ipynb', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'processing_status.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'data_investigation_scratch.ipynb', 'simply_load_and_inspect_data.ipynb', 'grouping_and_crossvalidation.ipynb', 'sample-entropy-numba-impl.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH
from predicament.utils.file_utils import drop_nan_cols
from predicament.utils.file_utils import load_featured_dataframe_and_config
from predicament.utils.file_utils import load_windowed_dataframe_and_config
from predicament.utils.file_utils import load_dataframe_and_config
from predicament.utils.file_utils import write_dataframe_and_config

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from predicament.data.features import filter_features

from prepare_evaluation_data import group_conditions

In [5]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

In [6]:
subdir = 'tiny'
df, config = load_featured_dataframe_and_config(subdir)
df

,participant,condition,window index,LempelZivEntropy[b=1][0],LempelZivEntropy[b=1][1],LempelZivEntropy[b=1][2],LempelZivEntropy[b=1][3]
0,VG_01,0,0,0.071391,0.082127,0.222119,0.319434
1,VG_01,0,1,0.102242,0.077912,0.204755,0.300513
2,VG_01,0,2,0.147764,0.098599,0.207703,0.282074
3,VG_01,0,3,0.101844,0.110798,0.254202,0.274748
4,VG_01,0,4,0.121624,0.067638,0.236958,0.294932
5,VG_01,0,5,0.114590,0.087197,0.247225,0.317747
6,VG_01,0,6,0.144632,0.103419,0.274939,0.260930
7,VG_01,0,7,0.084237,0.061797,0.231917,0.262980
8,VG_01,0,8,0.078020,0.067997,0.212160,0.276320
9,VG_01,0,9,0.087188,0.075730,0.165899,0.288155


In [10]:
subdir = 'tiny2'
df2, config2 = load_featured_dataframe_and_config(subdir)
df2

,participant,condition,window index,LempelZivEntropy[b=1][0],LempelZivEntropy[b=1][1],LempelZivEntropy[b=1][2],LempelZivEntropy[b=1][3]
0,VG_01,0,0,0.017403,0.021036,0.114402,0.140998
1,VG_01,0,1,0.020998,0.028269,0.105058,0.141045
2,VG_01,0,2,0.028186,0.032597,0.094321,0.121952
3,VG_01,0,3,0.042692,0.031777,0.091812,0.101795
4,VG_01,0,4,0.043462,0.031028,0.096320,0.102137
5,VG_01,0,5,0.036911,0.033862,0.087528,0.096534
6,VG_01,0,6,0.034481,0.030096,0.086178,0.085842
7,VG_01,0,7,0.032701,0.027199,0.083404,0.082898
8,VG_01,0,8,0.028596,0.025227,0.084903,0.075878
9,VG_01,0,9,0.032397,0.024709,0.068147,0.101669


In [7]:
config

{'DEFAULT': {},
 'LOAD': {'participant_list': ['VG_01',
   'VG_02',
   'VG_03',
   'VG_05',
   'VG_06',
   'VG_07',
   'VG_08',
   'VG_09',
   'VG_10',
   'VH_01',
   'VH_02',
   'VH_03'],
  'conditions': ['exper_video',
   'wildlife_video',
   'familiar_music',
   'tchaikovsky',
   'family_inter'],
  'channels': ['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG Fpz-F8'],
  'n_channels': 4,
  'sample_rate': 250,
  'window_size': 1024,
  'window_step': 128,
  'label_mapping': ['exper_video',
   'familiar_music',
   'family_inter',
   'tchaikovsky',
   'wildlife_video'],
  'data_format': 'dreem'},
 'WINDOWED': {'group_col': 'participant',
  'target_col': 'condition',
  'label_cols': ['participant', 'condition', 'window index']},
 'FEATURED': {'feature_set': ['LempelZivEntropy'],
  'feature_names': ['LempelZivEntropy[b=1][0]',
   'LempelZivEntropy[b=1][1]',
   'LempelZivEntropy[b=1][2]',
   'LempelZivEntropy[b=1][3]'],
  'removed_features': []}}

In [8]:
df['condition'].unique()



array([0])

In [9]:
string_ = "inactive:baseline,break;active:exper_video,wildlife_video,familiar_music,tchaikovsky,family_inter"
dict_ = get_dict_from_string(string_)
dict_

NameError: name 'get_dict_from_string' is not defined

In [ ]:
get_dict_of_lists_from_dict_of_strings(dict_)

In [24]:
from predicament.measures.lempel_ziv import lempel_ziv_entropy
from predicament.measures.lempel_ziv import lempel_ziv_entropy_numpy
import math

In [57]:
r1 = np.random.randint(0,3,30)

In [69]:


def contains(small, big):
    try:
        big.tobytes().index(small.tobytes())//big.itemsize
        return True
    except ValueError:
        return False


def lempel_ziv_entropy(l):
    """
    input
    -----
    l - a 1d numpy timeseries array with dtype=bool
    """
    n = len(l)
    sum_gamma = 0

    for i in range(1, n):
        sequence = l[:i]
        print(f"sequence = {sequence}")
        for j in range(i+1, n+1):
            s = l[i:j]
            if not contains(s, sequence):
                print(f"breaking with s = {s}")
                sum_gamma += len(s)
                break
    print(f"sum_gamma = {sum_gamma}")
    ae = 1 / (sum_gamma / n) * math.log(n)
    return ae

def lempel_ziv_entropy_set(l):
    """
    input
    -----
    l - a 1d numpy timeseries array with dtype=bool
    """
    n = len(l)
    sum_gamma = 0
    unique_sequences = set()
    unique_sequences.add(tuple(l[:1]))

    for i in range(1, n):
        sequence = l[:i]
        print(f"sequence = {sequence}")
        for j in range(i+1, n+1):
            s = tuple(l[i:j])
            if not s in unique_sequences:
                print(f"breaking with s = {s}")
                sum_gamma += len(s)
                unique_sequences.add(s)
                break
    print(f"sum_gamma = {sum_gamma}")
    ae = 1 / (sum_gamma / n) * math.log(n)
    return ae


## alternative implementation, but it has problems
def lempel_ziv_entropy_numpy(l):
    """
    input
    -----
    l - a 1d numpy timeseries array with dtype=bool
    """
    n = len(l)
    sum_gamma = 0
    for i in range(1, n):
        sequences = np.split(l[:i], np.where(np.diff(l[:i]))[0]+1)
        print(f"l[:i] = {l[:i]}")
        print(f"np.diff(l[:i]) = {np.diff(l[:i])}")
        print(f"np.where(np.diff(l[:i]))[0] = {np.where(np.diff(l[:i]))[0]}")
        print(f"sequences = {sequences}")
        sequence_lengths = [len(seq) for seq in sequences]
        max_sequence_length = max(sequence_lengths)
        sum_gamma += max_sequence_length
    ae = 1 / (sum_gamma / n) * np.log(n)
    return ae
    
## End of Lempel-Ziv entropy estimator



In [70]:
lempel_ziv_entropy(r1)

sequence = [1]
breaking with s = [1 2]
sequence = [1 1]
breaking with s = [2]
sequence = [1 1 2]
breaking with s = [0]
sequence = [1 1 2 0]
breaking with s = [1 2 0 1]
sequence = [1 1 2 0 1]
breaking with s = [2 0 1 0]
sequence = [1 1 2 0 1 2]
breaking with s = [0 1 0]
sequence = [1 1 2 0 1 2 0]
breaking with s = [1 0]
sequence = [1 1 2 0 1 2 0 1]
breaking with s = [0 0]
sequence = [1 1 2 0 1 2 0 1 0]
breaking with s = [0 1 2 1]
sequence = [1 1 2 0 1 2 0 1 0 0]
breaking with s = [1 2 1]
sequence = [1 1 2 0 1 2 0 1 0 0 1]
breaking with s = [2 1]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2]
breaking with s = [1 2 1]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1]
breaking with s = [2 1 2]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2]
breaking with s = [1 2 1 0]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1]
breaking with s = [2 1 0]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1 2]
breaking with s = [1 0 2]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1 2 1]
breaking with s = [0 2]
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1

1.2915939424033502

In [71]:
lempel_ziv_entropy_set(r1)

sequence = [1]
breaking with s = (1, 2)
sequence = [1 1]
breaking with s = (2,)
sequence = [1 1 2]
breaking with s = (0,)
sequence = [1 1 2 0]
breaking with s = (1, 2, 0)
sequence = [1 1 2 0 1]
breaking with s = (2, 0)
sequence = [1 1 2 0 1 2]
breaking with s = (0, 1)
sequence = [1 1 2 0 1 2 0]
breaking with s = (1, 0)
sequence = [1 1 2 0 1 2 0 1]
breaking with s = (0, 0)
sequence = [1 1 2 0 1 2 0 1 0]
breaking with s = (0, 1, 2)
sequence = [1 1 2 0 1 2 0 1 0 0]
breaking with s = (1, 2, 1)
sequence = [1 1 2 0 1 2 0 1 0 0 1]
breaking with s = (2, 1)
sequence = [1 1 2 0 1 2 0 1 0 0 1 2]
breaking with s = (1, 2, 1, 2)
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1]
breaking with s = (2, 1, 2)
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2]
breaking with s = (1, 2, 1, 0)
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1]
breaking with s = (2, 1, 0)
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1 2]
breaking with s = (1, 0, 2)
sequence = [1 1 2 0 1 2 0 1 0 0 1 2 1 2 1 2 1]
breaking with s = (0, 2)
sequence = [1 1 2 0 1 

1.2443405054861545

In [62]:
def lempel_ziv_entropy_cumsum(l):
    """
    input
    -----
    l - a 1d numpy timeseries array with dtype=bool
    """
    n = len(l)
    sum_gamma = 0

    for i in range(1, n):
        sequences = np.cumsum(l[:i], axis=0)
        print(f"sequences = {sequences}")
        unique_sequences = np.unique(sequences, axis=0)
        sum_gamma += len(unique_sequences)

    ae = 1 / (sum_gamma / n) * math.log(n)
    return ae

In [66]:
lempel_ziv_entropy_numpy(r1)

l[:i] = [1]
np.diff(l[:i]) = []
np.where(np.diff(l[:i]))[0] = []
sequences = [array([1])]
l[:i] = [1 1]
np.diff(l[:i]) = [0]
np.where(np.diff(l[:i]))[0] = []
sequences = [array([1, 1])]
l[:i] = [1 1 2]
np.diff(l[:i]) = [0 1]
np.where(np.diff(l[:i]))[0] = [1]
sequences = [array([1, 1]), array([2])]
l[:i] = [1 1 2 0]
np.diff(l[:i]) = [ 0  1 -2]
np.where(np.diff(l[:i]))[0] = [1 2]
sequences = [array([1, 1]), array([2]), array([0])]
l[:i] = [1 1 2 0 1]
np.diff(l[:i]) = [ 0  1 -2  1]
np.where(np.diff(l[:i]))[0] = [1 2 3]
sequences = [array([1, 1]), array([2]), array([0]), array([1])]
l[:i] = [1 1 2 0 1 2]
np.diff(l[:i]) = [ 0  1 -2  1  1]
np.where(np.diff(l[:i]))[0] = [1 2 3 4]
sequences = [array([1, 1]), array([2]), array([0]), array([1]), array([2])]
l[:i] = [1 1 2 0 1 2 0]
np.diff(l[:i]) = [ 0  1 -2  1  1 -2]
np.where(np.diff(l[:i]))[0] = [1 2 3 4 5]
sequences = [array([1, 1]), array([2]), array([0]), array([1]), array([2]), array([0])]
l[:i] = [1 1 2 0 1 2 0 1]
np.diff(l[:i]) = [ 0  1 -

1.214713350593627

In [49]:
lempel_ziv_entropy_cumsum(r1)

sequences = [1]
sequences = [1 3]
sequences = [1 3 5]
sequences = [1 3 5 5]
sequences = [1 3 5 5 7]
sequences = [1 3 5 5 7 9]
sequences = [ 1  3  5  5  7  9 11]
sequences = [ 1  3  5  5  7  9 11 13]
sequences = [ 1  3  5  5  7  9 11 13 13]
sequences = [ 1  3  5  5  7  9 11 13 13 14]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22 22]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22 22 24]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22 22 24 25]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22 22 24 25 25]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22 22 24 25 25 27]
sequences = [ 1  3  5  5  7  9 11 13 13 14 15 17 19 21 22 22 24 25 25 27 27]
sequences = [ 1  3  5  5  7  9 11 

0.3018814244078836

In [72]:
IDEAL_FEATURE_GROUP

NameError: name 'IDEAL_FEATURE_GROUP' is not defined